In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams["figure.dpi"] = 130
np.set_printoptions(suppress=True, linewidth=120, precision=4)
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 10)
pd.options.display.float_format = '{:,.2f}'.format

# --- Utility Functions ---
def note(msg, **kwargs):
    """Prints a formatted message with a notebook icon."""
    formatted_msg = textwrap.fill(msg, width=100, subsequent_indent='   ')
    print(f"\n📝 {formatted_msg}", **kwargs)
def sec(title):
    """Prints a formatted section title for code blocks."""
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note(f"Environment initialized. Pandas version: {pd.__version__}")

# Part 1: Foundations
## Chapter 1.13: Pandas: The Framework for Practical Data Analysis

### Introduction: Labeled, Relational Data for Economics

While NumPy provides the engine for high-performance numerical computing, it lacks features for handling labeled, heterogeneous data—the kind of messy, real-world data ubiquitous in economics. **Pandas** is the library that solves this problem. It provides two primary data structures, the `Series` (1D) and the `DataFrame` (2D), which are built on NumPy but add meaningful labels for rows and columns. 

This integration of data and labels, combined with a rich API for data manipulation (cleaning, merging, reshaping, aggregating), makes Pandas the indispensable tool for empirical analysis in Python. Whether you are cleaning survey data, analyzing financial time series, or preparing a dataset for an econometric model, Pandas provides the tools to do so efficiently and expressively.

### 1. The Pandas Data Model: `Series`, `DataFrame`, and `Index`

Understanding the internal structure of Pandas objects is key to using them effectively.
- **`Series`**: A 1D labeled array capable of holding any data type. It is essentially a NumPy array with an associated **`Index`**.
- **`DataFrame`**: A 2D labeled data structure with columns of potentially different types. It can be thought of as a dictionary of `Series` objects, all sharing the same `Index`.
- **`Index`**: The crucial, immutable array of labels for the rows. The index provides fast lookups and enables **alignment**—the automatic matching of data based on labels during operations. This is a cornerstone feature that prevents a huge class of common data manipulation errors.

In [ ]:
sec("DataFrame Structure and Automatic Alignment")
# Note the misaligned and missing labels
population = pd.Series({'USA': 331, 'DEU': 83, 'JPN': 126, 'CAN': 38}, name='Population (M)')
gdp = pd.Series({'USA': 25.46, 'DEU': 4.07, 'JPN': 4.23, 'FRA': 2.96}, name='GDP ($T)')

# The DataFrame aligns the data based on the shared index labels.
# 'CAN' and 'FRA' will have missing values (NaN) where they don't have a matching label.
df = pd.DataFrame({'Population': population, 'GDP': gdp})
display(df)

note("The DataFrame's Index, Columns, and underlying NumPy data:")
print(f"Index: {df.index}")
print(f"Columns: {df.columns}")
print(f"Underlying NumPy array:\n{df.values}")

### 2. Loading and Inspecting Data
The first step in any data analysis project is to load and inspect the data. We will use a dataset on the economic outcomes of different college majors, compiled by FiveThirtyEight.

In [ ]:
sec("Loading Data from a URL and Initial Inspection")
url = 'https://raw.githubusercontent.com/fivethirtyeight/data/master/college-majors/recent-grads.csv'
grads = pd.read_csv(url)

note("Displaying the first 3 rows with .head()")
display(grads.head(3))

note("Getting a concise summary of the DataFrame with .info()")
grads.info()

note("\nGenerating descriptive statistics with .describe()")
display(grads.describe())

### 3. Data Selection and Indexing: `.loc`, `.iloc`, and `[]`
Pandas provides a powerful and flexible indexing system. Using the explicit indexers is a best practice to write clear, unambiguous, and future-proof code.

| Method | Selects By... | Description |
| :--- | :--- | :--- |
| **`df.loc[rows, cols]`** | **Labels** | The primary method for selecting data. Slices are inclusive. |
| **`df.iloc[rows, cols]`** | **Integer Positions** | For when you need to select by position, like in NumPy. Slices are exclusive. |
| **`df[col]`** | **Column Label** | A convenient shorthand for selecting a single column (`Series`). |
| **`df[bool_series]`**| **Boolean Mask** | A convenient shorthand for filtering rows. |

**`loc` is the most versatile and is generally preferred.** It can take single labels, lists of labels, slices of labels, and boolean arrays.

#### The `SettingWithCopyWarning`
A common pitfall is **chained indexing** (e.g., `df[cols][rows]`). This can sometimes return a view and sometimes a copy of the data. Trying to assign a value to a copy will have no effect on the original DataFrame and will raise a `SettingWithCopyWarning`. **Always use `.loc` for assignment:** `df.loc[rows, cols] = value`.

In [ ]:
sec("Robust Data Selection with .loc")
grads_indexed = grads.set_index('Major')

note("Using .loc to select the 'ECONOMICS' row and specific columns:")
econ_data = grads_indexed.loc['ECONOMICS', ['Median', 'Unemployment_rate']]
display(econ_data)

note("\nUsing a boolean mask within .loc to find STEM majors with low unemployment:")
stem_categories = ['Engineering', 'Biology & Life Science', 'Computers & Mathematics', 'Physical Sciences']
is_stem = grads_indexed['Major_category'].isin(stem_categories)
low_unemployment = grads_indexed['Unemployment_rate'] < 0.04

result = grads_indexed.loc[is_stem & low_unemployment, ['Major_category', 'Median', 'Unemployment_rate']]
display(result.head())

### 4. Grouping and Aggregating Data
The **split-apply-combine** strategy is a core pattern for data analysis, implemented in Pandas with the `.groupby()` method. This process involves:
1.  **Split:** The data is split into groups based on a categorical variable (e.g., `Major_category`).
2.  **Apply:** A function is applied independently to each group (e.g., calculating the `mean` or `sum`).
3.  **Combine:** The results are combined back into a new DataFrame.

The `.agg()` method is the most powerful tool in this process, allowing you to apply multiple aggregation functions at once and to name the resulting columns.

In [ ]:
sec("Advanced Aggregation with .groupby() and .agg()")

note("Using .agg() to calculate multiple, named statistics for each major category:")
stats_by_cat = grads.groupby('Major_category').agg(
    Num_Majors=('Major', 'count'),
    Median_Salary=('Median', 'median'),
    Avg_Unemployment_Rate=('Unemployment_rate', 'mean'),
    Std_Dev_Salary=('Median', 'std')
).sort_values('Median_Salary', ascending=False)

display(stats_by_cat)

#### Advanced Grouping: `.transform` and `.filter`
Beyond aggregation, `groupby` offers two other powerful methods:
- **`.transform(func)`**: Applies a function to each group and returns a `Series` or `DataFrame` with the same shape as the original. This is ideal for group-wise operations like standardization (e.g., de-meaning a variable within each country group).
- **`.filter(func)`**: Subsets the original DataFrame, keeping only the groups for which the function returns `True`. This is useful for dropping groups that don't meet a certain criterion (e.g., groups with too few observations).

In [ ]:
sec("Group-wise Standardization with .transform")

note("De-meaning salary within each major category to see which majors over/underperform their category average.")
# The lambda function is applied to the 'Median' salary Series for each group.
demeaned_salary = grads.groupby('Major_category')['Median'].transform(lambda x: x - x.mean())

grads['Demeaned_Salary'] = demeaned_salary
display(grads.loc[grads['Major_category'] == 'Business', ['Major', 'Median', 'Demeaned_Salary']].head())

### 5. Merging and Joining Datasets
Combining data from different sources is a fundamental task. Pandas provides `pd.merge`, a high-performance, database-style join operation.

- `how='inner'` (default): Returns only the rows with matching keys in **both** DataFrames.
- `how='left'`: Returns all rows from the **left** DataFrame, and matched rows from the right. Unmatched rows in the right get `NaN`.
- `how='right'`: Returns all rows from the **right** DataFrame.
- `how='outer'`: Returns all rows from **both** DataFrames, filling `NaN` where there is no match.

In [ ]:
sec("Merging DataFrames")
# Create a hypothetical second DataFrame with category-level information
category_info = pd.DataFrame({
    'Major_category': ['Engineering', 'Business', 'Humanities & Liberal Arts', 'Arts'],
    'Difficulty': ['High', 'Medium', 'Medium', 'Low'],
    'Is_STEM': [True, False, False, False]
})

note("Original graduates data (first 3 rows):")
display(grads[['Major', 'Major_category', 'Median']].head(3))

note("\nHypothetical category info data:")
display(category_info)

# Merge the two DataFrames on the 'Major_category' column
grads_with_info = pd.merge(grads, category_info, on='Major_category', how='left')

note("\nResult of the left merge (first 5 rows):")
display(grads_with_info[['Major', 'Major_category', 'Median', 'Difficulty', 'Is_STEM']].head())

### 6. Reshaping Data: `stack`, `unstack`, and `pivot_table`

Data often needs to be reshaped between **long format** (where each observation is a row) and **wide format** (where each observation is spread across multiple columns).

- `.stack()`: Pivots a level of the column labels into the index, making the DataFrame taller (longer).
- `.unstack()`: Pivots a level of the row index into the column labels, making the DataFrame wider.
- `pd.pivot_table()`: A powerful and user-friendly tool for creating a spreadsheet-style pivot table. It's essentially a `groupby` operation followed by a reshape.

In [ ]:
sec("Using pivot_table to summarize data")

# Let's analyze the median salary by major category and gender.
# First, we need a 'Gender' column. For this example, let's create a dummy one.
rng = np.random.default_rng(42)
grads['Gender'] = rng.choice(['Men', 'Women'], size=len(grads), p=[0.52, 0.48])

note("Creating a pivot table to show median salary by category and gender:")
salary_pivot = pd.pivot_table(
    grads,
    values='Median', # The values to aggregate
    index='Major_category', # The rows of the pivot table
    columns='Gender', # The columns of the pivot table
    aggfunc='mean' # The aggregation function
)

display(salary_pivot.head())

### 7. Time Series Analysis
Pandas has first-class support for time series data, built around the `DatetimeIndex`. It provides a powerful toolkit for resampling, rolling window calculations, and other time-based operations.

In [ ]:
sec("Time Series Operations")
rng = np.random.default_rng(42)
dates = pd.date_range(start='2022-01-01', periods=365, freq='D')
data = rng.standard_normal(365).cumsum() + 100
ts = pd.Series(data, index=dates, name='StockPrice')

note("Slicing a time series with date strings:")
display(ts['2023-01-01':'2023-01-05'])

note("\nResampling daily data to monthly averages:")
monthly_avg = ts.resample('M').mean()
display(monthly_avg.head())

note("\nCalculating a 30-day rolling moving average and standard deviation:")
rolling_mean = ts.rolling(window=30).mean()
rolling_std = ts.rolling(window=30).std()

plt.figure(figsize=(11, 5))
ts.plot(label='Daily Price', style='-', alpha=0.6)
rolling_mean.plot(label='30-Day Rolling Mean', style='--', color='firebrick')
plt.fill_between(rolling_mean.index, 
                 rolling_mean - 2 * rolling_std, 
                 rolling_mean + 2 * rolling_std, 
                 color='gray', alpha=0.2, label='Bollinger Bands')
plt.legend(); plt.title('Stock Price and Rolling Statistics'); plt.show();

### 8. Writing Clean and Performant Pandas Code

#### 8.1 Method Chaining and `.pipe()`
A hallmark of professional Pandas code is the use of **method chaining** to create readable, linear data processing pipelines. Instead of creating many intermediate variables, you chain operations together. The `.pipe()` method is a key tool in this pattern, allowing you to cleanly insert any function that takes a DataFrame and returns a DataFrame into the chain.

In [ ]:
sec("Data Processing with Method Chaining")

def add_salary_range(df):
    """A function designed to be used in a .pipe() call."""
    df_out = df.copy()
    df_out['Salary_Range'] = df_out['P75th'] - df_out['P25th']
    return df_out

note("A data pipeline to find the top 5 majors with the largest salary range:")

top_5_range = (
    grads
    .dropna(subset=['P25th', 'P75th'])
    .pipe(add_salary_range)
    .sort_values('Salary_Range', ascending=False)
    .loc[:, ['Major', 'Major_category', 'P25th', 'P75th', 'Salary_Range']]
    .head(5)
    .reset_index(drop=True)
)

display(top_5_range)

#### 8.2 Using Categorical Data for Memory and Performance
Columns that have a low number of unique values (e.g., 'Major_category') are often stored as the `object` dtype, which means each entry is a full Python string object. This is very inefficient. Converting these columns to the `Categorical` dtype can lead to massive memory savings and performance improvements in operations like `groupby`.

A `Categorical` column stores the unique values once and then uses a simple integer array to represent the data, which is much more memory-friendly.

In [ ]:
sec("Memory Savings with Categorical Data")

note("Memory usage of the 'Major_category' column as an object dtype:")
mem_object = grads['Major_category'].memory_usage(deep=True)
print(f"  {mem_object / 1024:.2f} KB")

note("\nConverting to a Categorical dtype:")
grads['Major_category'] = grads['Major_category'].astype('category')
mem_categorical = grads['Major_category'].memory_usage(deep=True)
print(f"  Memory usage as categorical: {mem_categorical / 1024:.2f} KB")

note(f"\nMemory savings: {(1 - mem_categorical / mem_object):.1%}")

### 9. Exercises

1.  **Data Cleaning and Selection:** Using the `grads` DataFrame, first drop all rows that have missing values in the `Median` or `Unemployment_rate` columns (`.dropna()`). Then, using `.loc` with boolean indexing, select all majors where the median salary is greater than $60,000 and the unemployment rate is less than the overall average unemployment rate for the entire dataset.

2.  **Advanced Grouping:** Group the `grads` DataFrame by `Major_category`. Use `.agg()` to find the following for each category:
    a. The 10th percentile of `Median` salary.
    b. The 90th percentile of `Median` salary.
    c. The total number of `Men` and `Women` graduates (hint: `('Men', 'sum')`).
    Create a new column in the resulting DataFrame called `Salary_Range` which is the difference between the 90th and 10th percentile salaries.

3.  **Merging and Analysis:** Create a new DataFrame called `stem_status` with two columns: `Major_category` and `is_stem`. The `is_stem` column should be `True` for the categories 'Engineering', 'Biology & Life Science', 'Computers & Mathematics', 'Physical Sciences' and `False` otherwise. Merge this DataFrame with the original `grads` DataFrame. Finally, use `pivot_table` to calculate the average `Median` salary for STEM vs. non-STEM majors.

4.  **Time Series Analysis:** Create a time series of 500 daily stock prices with a starting value of 200 and daily returns drawn from a normal distribution with `mean=0.0001` and `std_dev=0.01`. 
    - Resample the data to get the last price of each month (`'M'`). This is the monthly closing price.
    - From the monthly data, calculate the monthly returns: `(price_t / price_{t-1}) - 1`.
    - Plot the monthly returns.

5.  **Reshaping with `melt`:** Start with the `salary_pivot` table you created in section 6. This is a "wide" DataFrame. Use `pd.melt()` to transform it into a "long" DataFrame with three columns: `Major_category`, `Gender`, and `Median_Salary`. This format is often required for plotting libraries like Seaborn or for econometric models.